In [1]:
import subprocess
import os
import glob
cwd = os.getcwd()


In [14]:
cwd_exe =os.path.join(cwd,'softBV0405.exe')
#cwd_cif =os.path.join(cwd[:-4],'test/CsCl.cif')
all_files = glob.glob(os.path.join(cwd[:-4],'test/Li_database_CN4_occ1/*.cif'))
all_files[0]

'd:\\Study\\softBV_mix\\GitHub\\projects\\Coord\\test/Li_database_CN4_occ1\\10062.cif'

In [16]:

process = subprocess.run([cwd_exe, "--cal-cn-bv", all_files[0], "Li1"], cwd=r'd:\Study\softBV_mix\GitHub\projects\Coord\bin', capture_output=True)
stdout = str(process.stdout, "utf-8")

In [17]:

print(stdout)

Message: reading cif file...
Message: total lines in the cif: 80
Message: determining symmetry from _symmetry_equiv_pos_as_xyz/_space_group_symop_operation_xyz ...
Message: calculated cell volume = 415.890852
Message: sf=0.750000
CN: Center atom:   Li1 (Li1+) 
CN: XYZ: [ -0.9395 1.7905 13.7560 ]
CN: Coordination Number:        4.000
CN: BVS: 0.9609      SD of BV:   0.033
CN: Index Coordinated type       r    occ       n       s     BVS   s_ave   s_det  [   X       Y       Z  ]
CN:     1              O2-  1.8420  1.000   1.000  0.2724  0.2724  0.2724  0.0000  -1.9373  3.2307 14.3249
CN:     2              O2-  1.9863  1.000   2.000  0.2060  0.4784  0.2392  1.3225   0.9423  1.5073 14.3249
CN:     3              O2-  2.0027  1.000   3.000  0.1995  0.6779  0.2260  1.1988  -1.7765  0.0624 14.3249
CN:     4              O2-  2.0332  1.000   4.000  0.1881  0.8659  0.2165  1.2015  -0.9423  1.6929 11.7251
CN:     5              O2-  3.1885  1.000   5.000  0.0200  0.8860  0.1772 10.8026  -0.8342

In [18]:
# nest list to store all the coordinates of the local polyhedral
Coor = [[]]
stdout_lines = stdout.split("\r\n")
for line in stdout_lines:
    line = line.split()
    if len(line) < 1: continue
    if line[1] == "Center":
        center = line[3]
    if line[1] == "XYZ:":
        Coor[0].append(center)
        Coor[0].append((float(line[3]),float(line[4]),float(line[5])))
    if line[1] == "Coordination":
        CN = float(line[3])
    if len(line) < 12: continue
    if line[5].isalpha(): continue
    if float(line[5]) <= CN:
        Coor.append([line[2]])
        Coor[-1].append((float(line[10]),float(line[11]),float(line[12])))

print(Coor)

[['Li1', (-0.9395, 1.7905, 13.756)], ['O2-', (-1.9373, 3.2307, 14.3249)], ['O2-', (0.9423, 1.5073, 14.3249)], ['O2-', (-1.7765, 0.0624, 14.3249)], ['O2-', (-0.9423, 1.6929, 11.7251)]]


In [47]:
# Calculate the angles
angles = []
l = len(Coor)
for i in range(1,l-1):
    for j in range(i+1,l):
        angles.append(calAngleDeg(Coor[0][1],Coor[i][1],Coor[j][1]))
angles.sort(reverse=True)

In [46]:
import numpy as np

def calAngleCos(V0,V1,V2):
    V0 = np.asarray(V0)
    V1 = np.asarray(V1)
    V2 = np.asarray(V2)
    Va = V1 - V0   
    Vb = V2 - V0
    cos = np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb))
    return round(cos, 4)
# Calculate the angle radians
def calAngle(V0,V1,V2):
    V0 = np.asarray(V0)
    V1 = np.asarray(V1)
    V2 = np.asarray(V2)
    Va = V1 - V0   
    Vb = V2 - V0
    dot_product = round(np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb)),4)
    angle = np.arccos(dot_product)
    return angle

def calAngleDeg(V0,V1,V2):
    V0, V1, V2 = [np.asarray(i) for i in [V0,V1,V2]]
    Va = V1 - V0   
    Vb = V2 - V0
    dot_product = round(np.dot(Va/np.linalg.norm(Va), Vb/np.linalg.norm(Vb)),4)
    angle = np.arccos(dot_product)
    deg = np.rad2deg(angle)
    deg = np.around(deg,3)  
    deg = deg.tolist()
    return deg


In [33]:
np.around([22.37, 33.64], decimals=1)


array([22.4, 33.6])